In [1]:
import os, re, time, requests,json
from bs4 import BeautifulSoup
from openai import OpenAI
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed  
import pandas as pd

# 爬取

## 爬取menet.com

In [5]:
host = 'https://www.menet.com.cn/news'

In [7]:
def get_list(host):
    response = requests.post(host)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_list = [_ for _ in soup.find_all('h3') if "登录" not in _.text]
    return soup, news_list

In [9]:
def get_info(item, host ='https://www.menet.com.cn'):
    link, title, date = host + item.a['href'], item.text,  item.a['href'].split('/')[3][:8]
    return {
    "title": title,
    'url': link,
    'date':date
}

In [11]:
soup, news_list = get_list(host)

In [12]:
result_menet = list(map(get_info, news_list))

In [15]:
for num, item in enumerate(soup.find_all(lambda tag: tag.has_attr('class') and len(tag)==1 and tag['class'] == ["item_time"])):
    result_menet[num]['date'] = item.text

In [17]:
for _ in result_menet: _['date'] = datetime.strptime(_['date'], '%Y-%m-%d').date()

---

## 爬取pharnexcloud

In [21]:
r = requests.get('https://www.pharnexcloud.com/zixun')

In [22]:
soup = BeautifulSoup(r.text, 'html.parser')

In [23]:
dates = soup.find_all(lambda tag:tag.has_attr('class') and tag['class']==['date'])
dates = [_.text for _ in dates]
dates = [_ for _  in dates if _ != '']
dates = [datetime.strptime(_.split(' ')[0], '%Y-%m-%d').date() for _ in dates]

In [24]:
result = soup.find_all('a', class_ = 'img-content')

In [25]:
def query_tilte_url(item):
    title = item.find_next()['alt']
    url = 'https://www.pharnexcloud.com' + item['href']
    return {
        'title': title,
        'url': url
    }

In [31]:
result_pharnexcloud = list(map(query_tilte_url, result))

In [33]:
for no, item in enumerate(result_pharnexcloud): item['date'] = dates[no]

---

## 爬取bydrug

In [37]:
r = requests.get('https://bydrug.pharmcube.com/')

In [38]:
soup = BeautifulSoup(r.text, 'html.parser')

In [39]:
datas = soup.find_all('div', class_= 'mf-flex mf-items-center')

In [43]:
datas = datas[1:]

In [45]:
item = datas[0]

In [47]:
def get_info(item):
    title = item.text.strip()
    url = item.find('a')['href']
    date = re.findall(r'\d{4}-\d{2}-\d{2}', requests.get(url).text) [0]
    date = datetime.strptime(date, '%Y-%m-%d').date()
    return {
        'title': title,
        'url': url,
        'date': date
    }

In [49]:
result_bydrug = [get_info(item) for _ in datas]

In [50]:
results = [*result_menet, *result_pharnexcloud, *result_bydrug]

---

In [52]:
def get_yesterday():
    import datetime  
  
    # 获取当前日期  
    today = datetime.date.today()  
      
    # 获取昨天的日期  
    yesterday = today - datetime.timedelta(days=1)  
      
    # 将昨天的日期格式化为“年-月-日”  
    return yesterday

In [58]:
results = [_ for _ in results if _['date'] == get_yesterday()]

In [88]:
results = [_[-1] for _ in {_['title']: _ for _ in results}.items()]

In [35]:
def summarize(url):
    context = requests.get('https://r.jina.ai/'+url).text
    url_pattern = re.compile(r'\bhttps?:\/\/[^\s/$.?#].[^\s]*\b')  
    cleaned_text = url_pattern.sub('', context)
    context = cleaned_text[:6000] if len(cleaned_text)>6000  else cleaned_text
    context = "请尽量简洁总结以下内容，并发表你的观点,文字不得超过150字\n"+context.replace("\n","")[:6000]
    client = OpenAI(
    api_key='sk-69c9bf69fa2b4d2985834d6dc17c0790', # 如何获取API Key：https://help.aliyun.com/zh/model-studio/developer-reference/get-api-key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )
    completion = client.chat.completions.create(
        model="qwen-max", # 更多模型请参见模型列表文档。
        messages=[{"role": "user", "content": context}]
    )
    print(completion.choices[0].message.content)
    
    return completion.choices[0].message.content

In [36]:
for _ in results:
   try:
        _['context'] = summarize(_['url'])
      
   except Exception as e:
       print(e)
   print('------'*20)

2024年8月，医药领域活跃，143款新药在国内获批临床，较上月增3款，抗肿瘤与免疫调节药物占主导。全球范围内，46款药物获得孤儿药等特殊资格认定，显示出药物研发加速的趋势。多家企业如拜耳、安进公布了积极的临床试验结果，涉及肿瘤、眼科及心力衰竭等领域。同时，创新药政策动态与研发进展TOP20亦反映出行业的蓬勃创新力。信达生物等企业的9款新药成功上市，标志着治疗方案的进一步丰富。药融云通过详实月报，助力行业跟踪全球新药动态。
------------------------------------------------------------------------------------------------------------------------
百济神州的新型PRMT5抑制剂BGB-58067片临床试验申请已获受理，该药物针对MTAP缺失的实体瘤，利用"合成致死"机制抗癌，预计2024年进入临床阶段。这标志着公司在抗癌药物研发领域的前沿探索又迈进一步，全球范围内已有10款同类药物进入临床。百济神州计划同年推进多个新分子实体的临床试验，彰显其在肿瘤治疗领域的广泛布局和强劲研发实力。
------------------------------------------------------------------------------------------------------------------------
扬子江药业的复方聚乙二醇电解质散（III）成功获批上市，预计市场潜力超过10亿元。该产品上市可能改变化学药品市场格局，为企业带来巨大商机。药融云提供多渠道服务，包括免费试用企业版、在线咨询、客服热线以及微信互动，助力行业人士深入了解药品研发动态与市场信息，同时推出生物医药个人版数据库，便于用户随时获取数据资源。
------------------------------------------------------------------------------------------------------------------------
科济药业2024中期业绩显示，公司虽净亏损收窄至3.52亿人民币，但收入增长，主要得益于其CAR-T产品赛恺泽®的销售。公司研发持续聚焦血液恶性肿瘤与实体瘤创新疗法，管线丰富，技术平台如Cy

In [ ]:
results

In [37]:
def get_flag(item):
    
    title = item['title']
    url = item['url']
    date = item['date'].strftime('%Y-%m-%d')
    context = item['context']
    return f"""
  <h4 style="color:black; margin-top:10px">  
    {title}  
  </h4>  
  
  <span> {date}</span>  
  <div  style="margin-bottom: 20px;">  
    <p>  
      {context}  
    </p>  
    <span><a href={url}>👉查看更多内容</a></span>  
  </div>  
  <hr />  
  
  
""".strip()

In [38]:
htmls = [get_flag(item) for item in results]

In [39]:
htmls = "".join(htmls)

In [40]:
def get_token():
    corp_id = 'ww9bffa3285527b3c8'
    secret = '_uOqAIKUWnRgFIY23gItQ5fX4OeWRpYu1G86Gvk7EUw'
    url = f'https://qyapi.weixin.qq.com/cgi-bin/gettoken?corpid={corp_id}&corpsecret={secret}'
    return requests.get(url).json()['access_token']

In [41]:
try:
    html_text = f"""  
<!DOCTYPE html>  
<html lang="en">  
  
<head>  
  <meta charset="UTF-8">  
  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">  
  <title>Document</title>  
</head>  
<style>  
  /* 全局样式 */  
  * {{  
    margin: 0;  
    padding: 0;  
  }}  
  
  h4 {{  
    text-align: center;  
     text-shadow: 2px 2px 4px rgba(15, 47, 206, 0.5);
  }}  
  
  em {{  
    font-family: 'Franklin Gothic Medium', 'Arial Narrow', Arial, sans-serif;  
    color: silver;  
  }}  
  
  div > p {{  
    width: 96%;  
    padding: 2%;  
  }}  
  
  a {{  
    color: #1e90ff;  
    text-decoration: none;  
    transition: color 0.3s;  
  }}  
  
  a:hover {{  
    color: #ff6347;  
  }}  
  
  p {{  
    text-indent: 2em;  
    text-align: justify;  
  }}  

  div {{
      margin-bottom: 10px;
  }}
</style>  
  
<body>  
  <h4 style="color:pink;">  
    {title}  
  </h4>  
  
  <span> 🗓️ {date}</span>  
  <div>  
    <p>  
      {context}  
    </p>  
    <span><a href={url}>👉查看更多内容</a></span>  
  </div>  
  <hr />  
  
</body>  
  
</html>  
"""
except:
    pass

In [42]:
first = f"""  
<!DOCTYPE html>  
<html lang="en">  
  
<head>  
  <meta charset="UTF-8">  
  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">  
  <title>Document</title>  
</head>  
<style>  
  /* 全局样式 */  
  * {{  
    margin: 0;  
    padding: 0;  
  }}  
  
  h4 {{  
    text-align: center;  
  }}  
  
  em {{  
    font-family: 'Franklin Gothic Medium', 'Arial Narrow', Arial, sans-serif;  
    color: silver;  
  }}  
  
  div > p {{  
    width: 96%;  
    padding: 2%;  
  }}  
  
  a {{  
    color: #1e90ff;  
    text-decoration: none;  
    transition: color 0.3s;  
  }}  
  
  a:hover {{  
    color: #ff6347;  
  }}  
  
  p {{  
    text-indent: 2em;  
    text-align: justify;  
  }}  
</style>  
  
<body>  
""".strip()

In [43]:
last = """</body>  
  
</html>  
""".strip()

In [44]:
html_text = first+htmls+last

In [45]:
html_text = html_text.replace('\n', '')

In [46]:
data = {
   "touser" : "@all",
   "toparty" : "PartyID1|PartyID2",
   "totag" : "TagID1 | TagID2",
   "msgtype" : "text",
   "agentid" : '1000002',
   "text" : {
       "content" : html_text
   },
   "safe":0,
   "enable_id_trans": 0,
   "enable_duplicate_check": 0,
   "duplicate_check_interval": 1800
}

In [47]:
from datetime import datetime

In [48]:
title  = datetime.now().strftime("%Y年%m月%d日") + '医疗行业资讯'

In [49]:
data = {
   "touser" : "chenweishi|SuBei|liqian9|YuanGuangJi", # 切换正式环境必须修改！
   
   "msgtype" : "mpnews",
   "agentid" : 1000002,
   "mpnews" : {
       "articles":[
           {
               "title": title, 
               "thumb_media_id": "2tNI-yo4L7-cQmsgW2drOBeFz51E_R5vHa_mdIdU2pBdFUEPklOjTTM_SevJplbmb4khjjhmEVh_kZDALSuo8aQ",
               "author": "大倩倩",
               
               "content": html_text,
               "digest": " "
            }
       ]
   },
   "safe":0,
   "enable_id_trans": 0,
   "enable_duplicate_check": 0,
   "duplicate_check_interval": 1800
}


In [50]:
requests.post("https://qyapi.weixin.qq.com/cgi-bin/message/send?access_token="+get_token(), data= json.dumps(data), ).json()

{'errcode': 0,
 'errmsg': 'ok',
 'msgid': 'mrVtVXE39it1tWVvd57npCY7Jv6hWamvZI0gnAIH5pUhD-Rprv4NR7WSlb6iHcbZJj1n6g1B7l8Bd49npUp8_A'}